In [1]:
cd /mnt/data

/mnt/data


In [2]:
! aws s3 sync s3://czbiohub-maca/10x_data/10X_P7_8/ 10X_P7_8/

In [3]:
! samtools


Program: samtools (Tools for alignments in the SAM format)
Version: 1.9 (using htslib 1.9)

Usage:   samtools <command> [options]

Commands:
  -- Indexing
     dict           create a sequence dictionary file
     faidx          index/extract FASTA
     fqidx          index/extract FASTQ
     index          index alignment

  -- Editing
     calmd          recalculate MD/NM tags and '=' bases
     fixmate        fix mate information
     reheader       replace BAM header
     targetcut      cut fosmid regions (for fosmid pool only)
     addreplacerg   adds or replaces RG tags
     markdup        mark duplicates

  -- File operations
     collate        shuffle and group alignments by name
     cat            concatenate BAMs
     merge          merge sorted alignments
     mpileup        multi-way pileup
     sort           sort alignment file
     split          splits a file by read group
     quickcheck     quickly check if SAM/BAM/CRAM file appears intact
     fastq          conve

In [4]:
ptprc_location = 'chr1:138062861-138175708'

In [5]:
! samtools view -bh 10X_P7_8/possorted_genome_bam.bam $ptprc_location > 10X_P7_8/lung_ptprc.bam

In [6]:
ls -lha 10X_P7_8/

total 37G
drwxrwxr-x 2 ubuntu ubuntu 4.0K Sep  6 19:10 ./
drwxr-xr-x 5 ubuntu root   4.0K Sep  7 21:31 ../
-rw-rw-r-- 1 ubuntu ubuntu  62M Aug 31  2017 10X_P7_8.mus.cell-gene.csv
-rw-rw-r-- 1 ubuntu ubuntu  19G Aug 31  2017 10X_P7_8.tgz
-rw-rw-r-- 1 ubuntu ubuntu  12K Sep  1  2017 barcodes.tsv
-rw-rw-r-- 1 ubuntu ubuntu 340K Sep  1  2017 genes.tsv
-rw-rw-r-- 1 ubuntu ubuntu 3.6M Sep  7 21:32 lung_ptprc.bam
-rw-rw-r-- 1 ubuntu ubuntu  69K Sep  6 19:07 lung_ptprc.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu 931K Sep  6 19:10 lung_ptprc.sig
-rw-rw-r-- 1 ubuntu ubuntu  14M Sep  1  2017 matrix.mtx
-rw-rw-r-- 1 ubuntu ubuntu  612 Aug 31  2017 metrics_summary.csv
-rw-rw-r-- 1 ubuntu ubuntu  19G Jun 19 06:16 possorted_genome_bam.bam
-rw-rw-r-- 1 ubuntu ubuntu 5.5M Jun 19 06:16 possorted_genome_bam.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu 9.2M Sep 16  2017 raw_gene_bc_matrices_h5.h5
-rw-rw-r-- 1 ubuntu ubuntu 2.6M Aug 31  2017 web_summary.html


In [7]:
! samtools index 10X_P7_8/lung_ptprc.bam

In [8]:
ksizes = 21, 27, 33, 51
protein = True
dna = True
seed = 42
track_abundance = True
scaled = 1000
num_hashes = 0
input_is_protein = False 
check_sequence = False

In [9]:
import pandas as pd
barcodes = set(pd.read_csv('/mnt/data/10X_P7_8/barcodes.tsv', squeeze=True))
genes = set(pd.read_csv('/mnt/data/10X_P7_8/genes.tsv', squeeze=True))
len(barcodes)

624

In [10]:
from sourmash import DEFAULT_SEED, MinHash, load_sbt_index, create_sbt_index
from sourmash import signature as sig
from sourmash import sourmash_args
from sourmash.logging import notify, error, print_results, set_quiet
from sourmash.sbtmh import SearchMinHashesFindBest, SigLeaf

from sourmash.sourmash_args import DEFAULT_LOAD_K
DEFAULT_COMPUTE_K = '21,31,51'

DEFAULT_N = 500
WATERMARK_SIZE = 10000
 
def make_minhashes():
    # one minhash for each ksize
    Elist = []
    for k in ksizes:
        if protein:
            E = MinHash(ksize=k, n=num_hashes,
                        is_protein=True,
                        track_abundance=track_abundance,
                        scaled=scaled,
                        seed=seed)
            Elist.append(E)
        if dna:
            E = MinHash(ksize=k, n=num_hashes,
                        is_protein=False,
                        track_abundance=track_abundance,
                        scaled=scaled,
                        seed=seed)
            Elist.append(E)
    return Elist

def add_seq(Elist, seq, input_is_protein, check_sequence):
    for E in Elist:
        if input_is_protein:
            E.add_protein(seq)
        else:
            E.add_sequence(seq, not check_sequence)

def build_siglist(Elist, filename, name=None):
    return [ sig.SourmashSignature(E, filename=filename,
                                   name=name) for E in Elist ]

def save_siglist(siglist, output_fp, filename=None):
    # save!
    if output_fp:
        sig.save_signatures(siglist, args.output)
    else:
        if filename is None:
            raise Exception("internal error, filename is None")
        with open(filename, 'w') as fp:
            sig.save_signatures(siglist, fp)
    notify('saved {} signature(s). Note: signature license is CC0.'.format(len(siglist)))

In [11]:
%load_ext line_profiler

In [ ]:

import pysam
from tqdm import tqdm
import itertools


bam_filename = '/mnt/data/10X_P7_8/lung_ptprc.bam'

def make_10x_signatures(bam_filename):
    output = bam_filename.replace('.bam', '.sig')

    bam_file = pysam.AlignmentFile(bam_filename, mode='rb')
    cell_seqs = {barcode: make_minhashes() for barcode in barcodes}

    for a in tqdm(bam_file):
        if (a.mapq == 255                                    # high quality mapping
            and a.has_tag('CB') and a.get_tag('CB') in barcodes  # in our set of barcodes,
    #         and a.has_tag('GN') and a.get_tag['GN'] in genes   # that maps to a single gene,
    #         and a.has_tag('RE') and a.get_tag('RE') == 'E'   # specifically to an exon,
            and a.has_tag('UB')):                            # and has a good UMI

            barcode = a.get_tag('CB')
    #         print(a)
            # if this isn't marked a duplicate, count it as a UMI
            if not a.is_duplicate:
    #             print(f"Adding {a.seq} to {barcode}")
                add_seq(cell_seqs[barcode], a.seq,
                                input_is_protein, check_sequence)
    cell_signatures = [build_siglist(seqs, filename=bam_filename, name=barcode) for barcode, seqs in cell_seqs.items()]
    signatures_flat = list(itertools.chain(*cell_signatures))
    save_siglist(signatures_flat, output_fp=False, filename=output)

%lprun -f make_10x_signatures make_10x_signatures(bam_filename)

65375it [00:09, 6823.24it/s] 

In [ ]:
make_10x_signatures('/mnt/data/10X_P7_8/possorted_genome_bam.bam')